<a href="https://colab.research.google.com/github/Osvaldo1417/Tarea-de-Riesgo-de-Mercado-SH/blob/master/Tarea_de_Riesgo_de_Mercado_SH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext rpy2.ipython

Paso de carga de librerías en R de todas las que se utilizarán

In [ ]:
%%R
install.packages("quantmod")
library(quantmod)

install.packages("data.table")
library(data.table)

install.packages("PerformanceAnalytics")
library("PerformanceAnalytics")

install.packages("Deriv")
library(Deriv)

library(dplyr)

con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

# Funciones

In [ ]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}

#función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 


wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
}


#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist < p ) ] ]  
  esc=w [ ranking [ which( plist < p ) ] ]  
  sum(loss*esc)/(sum(esc))
} 

genera_esc <- function(lamda,w0,n){
  p_esc=matrix(0,n,1)
  for (i in (1:n))
  {
    p_esc[i]=w0*lambda^(i-1)
  }
  return(p_esc)
}

# Carga de Datos

## Acciones y Divisas

In [ ]:
%%R
fval=as.Date("20220331",format="%Y%m%d") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo

acc_names <- c("America","Carso","Walmart")
div_names <- c("Euros","Libras","Dolares")

#ACCIONES Y DIVISAS
#Cargar los símbolos de yahoo finance para EQ
Sym_acciones<-c  ("AMXL.MX", "GCARSOA1.MX", "WALMEX.MX" )#tienen que ir en orden alfabético
pos_acc=c(-5000,1000,1200) #monto inicial invertido en acciones
#Cargar los símbolos de yahoo finance para FX
Sym_divisas<-c("EURUSD=X","GBPUSD=X","USDMXN=X" ) #tienen que ir en orden alfabético
pos_div=c(700,-600,1500) #monto inicial invertido en divisas
nh=3660 #días de historia

In [ ]:
%%R
#CARGA DE DATOS DE ACCIONES
pos=cbind(t(pos_div),t(pos_acc))
start_date=fval-nh #fecha inicial
#Creación del objeto para guardar los datos
acciones<-new.env()
divisas<-new.env()
#obtener los datos
getSymbols.yahoo(Sym_acciones,env=acciones,from=start_date,to=fval+1)
getSymbols.yahoo(Sym_divisas,env=divisas,from=start_date,to=fval)

#limpiarlos, alinearnos y quedarnos con el precio de cierre 
bt.prep(acciones,align='remove.na',fill.gaps = T)
bt.prep(divisas,align='remove.na',fill.gaps=T)

precios_acc = acciones$prices
precios_div = divisas$prices

precios_div=cbind(precios_div[,1]*precios_div[,3],
                     precios_div[,2]*precios_div[,3],precios_div[,3])

precios_acc_div = merge(precios_acc,precios_div,join = "inner")
tail(precios_acc_div)

           AMXL.MX GCARSOA1.MX WALMEX.MX EURUSD.X GBPUSD.X USDMXN.X
2022-03-24   20.91       75.61     78.56 22.26207 26.71136 20.22664
2022-03-25   21.00       75.77     78.59 22.09215 26.47899 20.06630
2022-03-28   20.72       75.81     79.83 21.99689 26.39227 20.03015
2022-03-29   20.70       74.29     78.58 22.08800 26.32673 20.09125
2022-03-30   20.75       73.53     80.34 22.14488 26.14256 19.96560
2022-03-31   21.12       74.41     81.92 22.18767 26.11766 19.87815


In [ ]:
%%R
aux2=data.table(Date=as.Date(index(precios_acc_div)),coredata(precios_acc_div))
colnames(aux2) <- c("Date","America","Carso","Walmart","Euros","Libras","Dolares")
tail(aux2)

         Date America Carso Walmart    Euros   Libras  Dolares
1: 2022-03-24   20.91 75.61   78.56 22.26207 26.71136 20.22664
2: 2022-03-25   21.00 75.77   78.59 22.09215 26.47899 20.06630
3: 2022-03-28   20.72 75.81   79.83 21.99689 26.39227 20.03015
4: 2022-03-29   20.70 74.29   78.58 22.08800 26.32673 20.09125
5: 2022-03-30   20.75 73.53   80.34 22.14488 26.14256 19.96560
6: 2022-03-31   21.12 74.41   81.92 22.18767 26.11766 19.87815


## Bonos

In [ ]:
#CETES
base="RiesgosFinancieros/2020-2/Insumos/tasa_guber.txt"

#BONDES D
btasadescst="RiesgosFinancieros/2020-2/Insumos/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2020-2/Insumos/tfondeo.txt"
plazos_bdm=cbind( 358,	405,	550,	1200,	1800) #Vencimiento del bono
plazocupon_bdm=cbind( 28,	28, 28, 28, 28) #plazos_bdm fijos de cada cupón
contratos_bdm=cbind(220, -290, 290, -460, 100) #posición invertida
nominal_bdm=100

## Futuros

## Swap's

## Opciones

# Integración de insumos

In [ ]:
%%R
fechas= data.table(Date=as.Date(aux2[aux2,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity
tail(fechas)

         Date
1: 2022-03-24
2: 2022-03-25
3: 2022-03-28
4: 2022-03-29
5: 2022-03-30
6: 2022-03-31


## Segreración de Datos

### Acciones y Divisas

In [ ]:
%%R
n=nrow(fechas) #Historia de todos

#historia de acciones y divisas
precios_acc_div = fechas[aux2,on=.(Date),nomatch=0][order(-Date)]
precios_acc_div = precios_acc_div%>%select(-Date)
precios_acc_div

      America Carso Walmart    Euros   Libras  Dolares
   1:   21.12 74.41   81.92 22.18767 26.11766 19.87815
   2:   20.75 73.53   80.34 22.14488 26.14256 19.96560
   3:   20.70 74.29   78.58 22.08800 26.32673 20.09125
   4:   20.72 75.81   79.83 21.99689 26.39227 20.03015
   5:   21.00 75.77   78.59 22.09215 26.47899 20.06630
  ---                                                 
2511:   15.80 40.00   43.00 17.04491 20.32820 12.79050
2512:   15.73 39.97   41.58 16.99530 20.34285 12.75090
2513:   15.68 39.99   41.79 16.97394 20.27933 12.70500
2514:   15.63 40.14   41.49 16.80676 20.09019 12.65280
2515:   15.31 39.94   41.77 16.81659 20.15701 12.73520


### Bonos

### Futuros

### Swap's

### Opciones

# Simulación Historica

## Acciones y Divisas

In [ ]:
%%R
x0_acc_div=precios_acc_div[1,]
DeltaX_acc_div=as.matrix(log(precios_acc_div[1:(n-1)]/precios_acc_div[2:(n)]))
V0_acc_div=cbind(t(pos_acc),t(pos_div))*x0_acc_div
m_acc=length(pos_acc)
m_div=length(pos_div)

print("Precio Actual")
print(x0_acc_div)

print("Valor del portafolio")
print(V0_acc_div)

[1] "Precio Actual"
   America Carso Walmart    Euros   Libras  Dolares
1:   21.12 74.41   81.92 22.18767 26.11766 19.87815
[1] "Valor del portafolio"
   America Carso Walmart    Euros   Libras  Dolares
1: -105600 74410   98304 15531.37 -15670.6 29817.23


## Bonos

## Futuros

## Swap's

## Opciones

# Medición de Riesgo


## Acciones y Divisas

In [ ]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 8 instrumentos financieros (9 si separamos acciones y divisas)
n_if=matrix(0,6,1)
n_if[1]=ncol(precios_acc_div) #acciones y divisas
#n_if[2]=ncol(X_bd_ext) #bonde
#n_if[3]=ncol(X_sw) #swaps
#n_if[4]=ncol(X_oir) #opciones tasa de interés
#n_if[5]=ncol(X_futtdc) #Forwards de tipo de cambio
#n_if[6]=ncol(X_futind) #Forwards de índices
rownames(n_if) <- c("Acciones y Divisas","Bonos","Futuros TDC","Futuros IPC","Swap's","Opciones")

#valor del portafolios

#V0_port=cbind(V0_acc_div, V0_bd, V0_sw, V0_oir,V0_fwtdc,V0_fwind)
V0_port=cbind(V0_acc_div)
V0T_port=sum(V0_port)


#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
#X_port=cbind(precios_acc_div,X_bd_ext,X_sw,X_oir,X_futtdc,X_futind) #Factores de riesgo del portafolios de 8(9) instrumentos financieros
X_port=cbind(precios_acc_div) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1)]/X_port[2:(n)]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na


Ns=nrow(DeltaX_port) #Definimos número de escenarios históricos
#alpha=0.98 #Nivel de Confianza para las medidas de riesgo

DeltaX_s=DeltaX_port
print("Matriz de Rendimientos del Portafolio")
print(head(DeltaX_s))
print("Número de Instrumentos")
print(ncol(DeltaX_s))
print("Número de Datos")
print(nrow(DeltaX_s))
print("Número de instrumentos por Factores de Riesgo")
print(n_if)

[1] "Matriz de Rendimientos del Portafolio"
           America         Carso       Walmart        Euros        Libras
[1,]  0.0176742595  0.0118969226  0.0194755577  0.001930412 -0.0009526159
[2,]  0.0024124981 -0.0102828940  0.0221503396  0.002572132 -0.0070202441
[3,] -0.0009656205 -0.0202538171 -0.0157821591  0.004133322 -0.0024862505
[4,] -0.0134230686  0.0005277873  0.0156549847 -0.004321403 -0.0032804763
[5,]  0.0042949243  0.0021138336  0.0003817754 -0.007661797 -0.0087375911
[6,]  0.0086455870  0.0254501012  0.0072820509 -0.004376409 -0.0068161484
          Dolares
[1,] -0.004389554
[2,] -0.006273605
[3,]  0.003045759
[4,] -0.001803153
[5,] -0.007958856
[6,] -0.002249892
[1] "Número de Instrumentos"
[1] 6
[1] "Número de Datos"
[1] 2514
[1] "Número de instrumentos por Factores de Riesgo"
                   [,1]
Acciones y Divisas    6
Bonos                 0
Futuros TDC           0
Futuros IPC           0
Swap's                0
Opciones              0


In [ ]:
%%R
m=m_acc+m_div    #PASO CLAVE
X_s_acc_div=matrix(0,Ns,m)
V_acc_div=matrix(0,Ns,m)
Vfr1_acc_div=matrix(0,Ns,m_acc)
Vfr2_acc_div=matrix(0,Ns,m_div)
PG_acc_div=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_acc_div=matrix(0,Ns,m_acc)
PGfr2_acc_div=matrix(0,Ns,m_div)
PGT_acc_div=matrix(0,Ns,1)
PGfr1T_acc_div=matrix(0,Ns,1)
PGfr2T_acc_div=matrix(0,Ns,1)

In [ ]:
%%R
DeltaX_s_acc_div=DeltaX_s[,(1:m)]
x0_acc_div=precios_acc_div[1,]

for (i in 1:Ns)
{
  X_s_acc_div[i,]=as.matrix(x0_acc_div*exp(DeltaX_s_acc_div[i,]))
  V_acc_div[i,]=cbind(t(pos_acc),t(pos_div))*X_s_acc_div[i,]

  Vfr1_acc_div[i,]=t(pos_acc)*X_s_acc_div[i,1:m_acc]
  Vfr2_acc_div[i,]=t(pos_div)*X_s_acc_div[i,(m_acc+1):m]

  PG_acc_div[i,]=as.matrix(V_acc_div[i,]-V0_acc_div)

  PGfr1_acc_div[i,]=as.matrix(Vfr1_acc_div[i,]-V0_acc_div[,1:m_acc])
  PGfr2_acc_div[i,]=as.matrix(Vfr2_acc_div[i,]-V0_acc_div[,(m_acc+1):m])

  PGT_acc_div[i,]=sum(PG_acc_div[i,])

  PGfr1T_acc_div[i,]=sum(PGfr1_acc_div[i,])
  PGfr2T_acc_div[i,]=sum(PGfr2_acc_div[i,])
}

In [ ]:
%%R
print("Perdida/Ganancias de acciones y divisas")
print(head(PG_acc_div))
print("Perdidas/Ganancias de acciones")
print(head(PGfr1_acc_div))
print("Perdidas/Ganancias de divisas")
print(head(PGfr2_acc_div))
print("Perdidas/Ganancias totales de acciones y divisas")
print(head(PGT_acc_div))
print("Perdidas/Ganancias totales de acciones")
print(head(PGfr1T_acc_div))
print("Perdidas/Ganancias totales de divisas")
print(head(PGfr2T_acc_div))

[1] "Perdida/Ganancias de acciones y divisas"
           [,1]        [,2]        [,3]       [,4]      [,5]       [,6]
[1,] -1882.9931   890.53688  1933.29002   30.01090  14.92095 -130.59748
[2,]  -255.0674  -761.22966  2201.76179   40.00016 109.62618 -186.47594
[3,]   101.9203 -1491.92700 -1539.27087   64.32902  38.91264   90.95452
[4,]  1408.0051    39.28302  1551.05679  -66.97251  51.32280  -53.71657
[5,]  -454.5194   157.45672    37.53721 -118.54352 136.32683 -236.36914
[6,]  -916.9320  1918.04583   718.46750  -67.82311 106.44992  -67.01014
[1] "Perdidas/Ganancias de acciones"
           [,1]        [,2]        [,3]
[1,] -1882.9931   890.53688  1933.29002
[2,]  -255.0674  -761.22966  2201.76179
[3,]   101.9203 -1491.92700 -1539.27087
[4,]  1408.0051    39.28302  1551.05679
[5,]  -454.5194   157.45672    37.53721
[6,]  -916.9320  1918.04583   718.46750
[1] "Perdidas/Ganancias de divisas"
           [,1]      [,2]       [,3]
[1,]   30.01090  14.92095 -130.59748
[2,]   40.00016 109.626

In [ ]:
%%R
w0=1-alpha
n = Ns
lambda =uniroot(function(x) w0*(1-x^(n))/(1-x)-1, c(0,0.99), tol = 1e-28)$root

p_esc=genera_esc(lambda,w0,n)
print("Pesos para el Alisado")
print(head(p_esc))
print(tail(p_esc))
print("Validación de los pesos")
sum(p_esc) #validamos que sume 1

[1] "Pesos para el Alisado"
           [,1]
[1,] 0.02000000
[2,] 0.01960000
[3,] 0.01920800
[4,] 0.01882384
[5,] 0.01844736
[6,] 0.01807842
                [,1]
[2509,] 1.977096e-24
[2510,] 1.937554e-24
[2511,] 1.898803e-24
[2512,] 1.860827e-24
[2513,] 1.823611e-24
[2514,] 1.787139e-24
[1] "Validación de los pesos"
[1] 1


In [ ]:
%%R
#VaR por posición
VaRCont_acc_div=matrix(0,1,m)
VaRfr1_acc_div=matrix(0,1,m_acc)
VaRfr2_acc_div=matrix(0,1,m_div)

CVaRCont_acc_div=matrix(0,1,m)
CVaRfr1_acc_div=matrix(0,1,m_acc)
CVaRfr2_acc_div=matrix(0,1,m_div)

VaRCont_CA_acc_div=matrix(0,1,m)
VaRfr1_CA_acc_div=matrix(0,1,m_acc)
VaRfr2_CA_acc_div=matrix(0,1,m_div)

CVaRCont_CA_acc_div=matrix(0,1,m)
CVaRfr1_CA_acc_div=matrix(0,1,m_acc)
CVaRfr2_CA_acc_div=matrix(0,1,m_div)

In [ ]:
%%R

          [,1]      [,2]      [,3]
[1,] -3349.254 -3017.479 -3146.197


In [ ]:
%%R
for (i in (1:m))
{
  #VaR de accines y divisas sin alisado
  VaRCont_acc_div[i]=equantile(PG_acc_div[,i],1-alpha,Ns)
  #CVaR de acciones y divisas sin alisado
  CVaRCont_acc_div[i]= mean(merge(which(PG_acc_div[,i]<VaRCont_acc_div[i]),cbind(seq(1,Ns),PG_acc_div[,i]), by.x=1,by.y=1)[,2])
  #VaR de acciones y divisas con alisado
  VaRCont_CA_acc_div[i]=wquantile(PG_acc_div[,i],p_esc, 1-alpha)
  #CVaR de acciones y divisas con alisado
  CVaRCont_CA_acc_div[i]= wcvar(PG_acc_div[,i],p_esc, 1-alpha)
}

#VaR por acciones
VaRfr1_acc_div <- as.matrix(t(VaRCont_acc_div[1:m_acc]))
#VaR por divisas
VaRfr2_acc_div <- as.matrix(t(VaRCont_acc_div[(m_acc+1):m]))

#CVar por acciones
CVaRfr1_acc_div <- as.matrix(t(CVaRCont_acc_div[1:m_acc]))
#CVar por divisas
CVaRfr2_acc_div <- as.matrix(t(CVaRCont_acc_div[(m_acc+1):m]))

#Var con alisado por acciones
VaRfr1_CA_acc_div <- as.matrix(t(VaRCont_CA_acc_div[1:m_acc]))
#Var con alsiado por divisas
VaRfr2_CA_acc_div <- as.matrix(t(VaRCont_CA_acc_div[(m_acc+1):m]))

#CVar con alisado por acciones
CVaRfr1_CA_acc_div <- as.matrix(t(CVaRCont_CA_acc_div[1:m_acc]))
#CVar con alisado por divisas
CVaRfr2_CA_acc_div <- as.matrix(t(CVaRCont_CA_acc_div[(m_acc+1):m]))

#Var sin alisado total
VaRTotal_acc_div=equantile(PGT_acc_div,1-alpha,Ns)
VaRTotalfr1_acc_div=equantile(PGfr1T_acc_div,1-alpha,Ns)
VaRTotalfr2_acc_div=equantile(PGfr2T_acc_div,1-alpha,Ns)

#CVar sin alisado total
CVaRTotal_acc_div= mean(merge(which(PGT_acc_div<VaRTotal_acc_div),cbind(seq(1,Ns),PGT_acc_div), by.x=1,by.y=1)[,2])
CVaRTotalfr1_acc_div= mean(PGfr1T_acc_div[which(PGfr1T_acc_div<VaRTotalfr1_acc_div),])
CVaRTotalfr2_acc_div= mean(PGfr2T_acc_div[which(PGfr2T_acc_div<VaRTotalfr2_acc_div),])

#Var con alisado total
VaRTotal_CA_acc_div=wquantile(PGT_acc_div,p_esc,1-alpha)
VaRTotalfr1_CA_acc_div=wquantile(PGfr1T_acc_div,p_esc,1-alpha)
VaRTotalfr2_CA_acc_div=wquantile(PGfr2T_acc_div,p_esc,1-alpha)

#CVar con alisado total
CVaRTotal_CA_acc_div= wcvar(PGT_acc_div,p_esc,1-alpha)
CVaRTotalfr1_CA_acc_div= wcvar(PGfr1T_acc_div,p_esc,1-alpha)
CVaRTotalfr2_CA_acc_div= wcvar(PGfr2T_acc_div,p_esc,1-alpha)

### Valor en Riesgo

In [ ]:
%%R
colnames(VaRCont_acc_div) <- c(acc_names,div_names)
colnames(VaRfr1_acc_div) <- acc_names
colnames(VaRfr2_acc_div) <- div_names
colnames(VaRCont_CA_acc_div) <- c(acc_names,div_names)
colnames(VaRfr1_CA_acc_div) <- acc_names
colnames(VaRfr2_CA_acc_div) <- div_names

colnames(CVaRCont_acc_div) <- c(acc_names,div_names)
colnames(CVaRfr1_acc_div) <- acc_names
colnames(CVaRfr2_acc_div) <- div_names
colnames(CVaRCont_CA_acc_div) <- c(acc_names,div_names)
colnames(CVaRfr1_CA_acc_div) <- acc_names
colnames(CVaRfr2_CA_acc_div) <- div_names

In [ ]:
%%R
print("VaR por acciones y divisas")
VaR_acc_div <- as.matrix(rbind(VaRCont_acc_div,VaRCont_CA_acc_div,V0_acc_div))
rownames(VaR_acc_div) <- c("Sin alisado","Con alisado","Valor portafolio")
print(VaR_acc_div)

print("VaR Total")
VaR_Total_acc_div <- as.matrix(rbind(cbind(VaRTotal_acc_div,VaRTotalfr1_acc_div,VaRTotalfr2_acc_div),
                                     cbind(VaRTotal_CA_acc_div,VaRTotalfr1_CA_acc_div,VaRTotalfr2_CA_acc_div),
                                     cbind(sum(V0_acc_div),sum(V0_acc_div[,1:m_acc]),sum(V0_acc_div[,(m_acc+1):m]))))
rownames(VaR_Total_acc_div) <- c("Sin alisado","Con alisado","Valor portafolio")
colnames(VaR_Total_acc_div) <- c("Total","Acciones","Divisas")
print(VaR_Total_acc_div)

[1] "VaR por acciones y divisas"
                     America     Carso   Walmart      Euros      Libras
Sin alisado        -3349.254 -3017.479 -3146.197  -292.0318   -282.2936
Con alisado        -3487.312 -2990.556 -3177.200  -156.9451   -134.5073
Valor portafolio -105600.005 74410.004 98303.998 15531.3719 -15670.5984
                    Dolares
Sin alisado       -453.4187
Con alisado       -356.7110
Valor portafolio 29817.2265
[1] "VaR Total"
                     Total  Acciones    Divisas
Sin alisado      -4571.742 -4704.415  -503.0257
Con alisado      -4949.976 -4973.593  -379.4284
Valor portafolio 96791.997 67113.997 29677.9999


### Valor en Riesgo Condicional

In [ ]:
%%R
print("CVaR por acciones y divisas")
CVaR_acc_div <- as.matrix(rbind(CVaRCont_acc_div,CVaRCont_CA_acc_div,V0_acc_div))
rownames(CVaR_acc_div) <- c("Sin alisado","Con alisado","Valor portafolio")
print(CVaR_acc_div)

print("CVaR Total")
CVaR_Total_acc_div <- as.matrix(rbind(cbind(CVaRTotal_acc_div,CVaRTotalfr1_acc_div,CVaRTotalfr2_acc_div),
                                     cbind(CVaRTotal_CA_acc_div,CVaRTotalfr1_CA_acc_div,CVaRTotalfr2_CA_acc_div),
                                     cbind(sum(V0_acc_div),sum(V0_acc_div[,1:m_acc]),sum(V0_acc_div[,(m_acc+1):m]))))
rownames(CVaR_Total_acc_div) <- c("Sin alisado","Con alisado","Valor portafolio")
colnames(CVaR_Total_acc_div) <- c("Total","Acciones","Divisas")
print(CVaR_Total_acc_div)

[1] "CVaR por acciones y divisas"
                     America     Carso   Walmart      Euros      Libras
Sin alisado        -4528.041 -4166.973 -4428.774  -356.0485   -419.0875
Con alisado        -4024.923 -3692.181 -3434.240  -215.0101   -164.5758
Valor portafolio -105600.005 74410.004 98303.998 15531.3719 -15670.5984
                    Dolares
Sin alisado       -598.4633
Con alisado       -578.8262
Valor portafolio 29817.2265
[1] "CVaR Total"
                     Total  Acciones    Divisas
Sin alisado      -6341.516 -6511.754  -638.6302
Con alisado      -6050.123 -6090.790  -473.4383
Valor portafolio 96791.997 67113.997 29677.9999
